# CLV-Calculation in Non-Contractual Business

The goal of this notebook is to explore (part) of the functionality of the [Lifetimes](https://github.com/CamDavidsonPilon/lifetimes) package. The pre-cleaned data is transformed, split into calibration and holdout sets and CLV is then calculated in two steps:
1. Prediction of future purchases with BG/NBD model
2. Prediction of monetary value of these future purchases with Gamma-Gamma model.

Because the results were not so good (maybe partly due to the seasonality in the underlying data, see notebook 1) a second run was made with outliers removed. But this worsened the results.

#### Data Source
- `data/interim/clv_data.csv`: Cleaned data, prepared in first notebook.

#### Changes
- 19-02-12: Start notebook
- 19-02-15: Calculate CLVs
- 19-02-17: add 2nd run with outliers removed

---

### Import libraries, load data

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import datetime as dt

from lifetimes.plotting import *
from lifetimes.utils import *
from lifetimes.estimation import *
from lifetimes import BetaGeoFitter

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(), sns.set_style('whitegrid')
%matplotlib inline  

# Special settings for lifetime plots
sns.set_palette("viridis")
sns.set(rc={'image.cmap': 'viridis'})

import cleaning_functions as clean
import EDA_functions as EDA

# Display Settings
from IPython.display import display
pd.options.display.max_columns = 100

IndentationError: unexpected indent (cleaning_functions.py, line 230)

In [ ]:
# Load data
clv_data = pd.read_csv('data/interim/clv_data.csv', parse_dates=['InvoiceDate'],
                       dtype={'CustomerID' : str, 'Country' : 'category'})

## 1) Prepare Data


In [ ]:
clv_data.info()

**Note:** For Lifetimes we need an Input Dataframe of transaction data of the form: 
- customer_id
- datetime 
- [monetary_value] (optional, but needed for clv estimations)

### Clean & Transform Data - with holdout set

(see notebook 'basic_functionalities' for transformation without holdout set)

In [ ]:
"""Transform data to 'Lifetimes' format"""

clv_data.drop(['Quantity', 'UnitPrice', 'Country'], inplace=True, axis=1)
clv_data['InvoiceDate'] = clv_data['InvoiceDate'].dt.date

# Use transformation function from liftimes package
clv_cal_holdout = calibration_and_holdout_data(clv_data, 'CustomerID', 'InvoiceDate', 
                                                   monetary_value_col='Sales',
                                                   calibration_period_end='2011-07-09',  # 7 months
                                                   observation_period_end='2011-12-09',  # 5 months
                                                   ) 

**IMPORTANT NOTE:** I tried to fit with a longer cal_period / shorter holdout_period, the fit was poorer than with this configuration. It seems important to experiment with the duration and then assess the fit (see lineplot below).

In [ ]:
# Check results
print(clv_cal_holdout.shape)
clv_cal_holdout.head()

**Nomenclature for the CLV model:**

- _Frequency_: represents the number of repeat purchases the customer has made. This means that it’s one less than the total number of purchases. (Thus if they have made only 1 purchase, the recency is 0.)
- _T_: represents the age of the customer in whatever time units chosen (daily, in our dataset). This is equal to the duration between a customer’s first purchase and the end of the period under study.
- _Recency_: represents the age of the customer when they made their most recent purchases. This is equal to the duration between a customer’s first purchase and their latest purchase. (Thus if they have made only 1 purchase, the recency is 0.)

**Note:** Only 2800 customers are in the fitting set, because the others made their first order only in the holdout period. (I checked that with help of the original 'customers_data'.)

In [ ]:
# 'Repair' duration_holdout: set to float - should not happen according to docs ...
import re
clv_cal_holdout['duration_holdout'] = \
    clv_cal_holdout['duration_holdout'].astype(str)    
clv_cal_holdout['duration_holdout'] = \
    clv_cal_holdout['duration_holdout'].apply(lambda x: int(re.findall('\d+', x)[0]))

In [ ]:
# Check results
display(clv_cal_holdout.head())

In [ ]:
# Check distributions
EDA.plot_num_hist(clv_cal_holdout, figsize=(16,8))

In [ ]:
print("prop of customers without repeat purchase in cal_period: ", \
        round(clv_cal_holdout['frequency_cal'].value_counts()[0] / len(clv_cal_holdout),3))

**Note:** 46.3% of customers are 'one-timers' in the cal_period. this could well be a problem for fitting the model.

## 2) Forecast number of future purchases with BG / NBD model
The BG /NBD model allows us to compute the expected number of purchases in a forecast period at the customer level. 

class BetaGeoFitter(). This model has the following assumptions:
1. Each individual, `i`, has a hidden `lambda_i` and `p_i` parameter
2. These come from a population wide Gamma and a Beta distribution respectively.
3. Individuals purchases follow a Poisson process with rate `lambda_i*t` .
4. After each purchase, an individual has a p_i probability of dieing (never buying again).

In [ ]:
# Fit on the _cal columns, and test on the _holdout columns"""
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(clv_cal_holdout['frequency_cal'], 
        clv_cal_holdout['recency_cal'], 
        clv_cal_holdout['T_cal'])

In [ ]:
# Plot results
plot_calibration_purchases_vs_holdout_purchases(bgf, 
                                                clv_cal_holdout, 
                                                figsize=(12,6),
                                                n=int(clv_cal_holdout['frequency_cal'].max() + 1),
                                                color=['rebeccapurple', 'yellow']);

In [ ]:
# Assess model fit (with simulated data, cal_period only)
plot_period_transactions(bgf, color=['rebeccapurple', 'yellow']);

### Calculate and Evaluate Individual Customer Predictions
Based on customer history, we can predict how many future purchases an individual might make in a given period.

In [ ]:
# Prepare dataset for all customers to predict on
clv = summary_data_from_transaction_data(clv_data, 'CustomerID', 'InvoiceDate', monetary_value_col='Sales')
# check results
print(clv.shape)
clv.head()

In [ ]:
def predict_individual_purchases(model, df, t):
    """Predict number of purchases for period t for each customer
    and return them, rounded to int, in a DataFrame.
    """
    pred_list = []
    for customer in df.itertuples():
        pred_purchases = model.predict(t, 
                                     int(customer[1]), 
                                     int(customer[2]), 
                                     int(customer[3])
                                     )
        pred_list.append({'id': customer[0], 
                          'pred_purchases': int(pred_purchases)})  # round to int

    pred_df = pd.DataFrame(pred_list, columns=['id', 'pred_purchases'])
    pred_df.set_index('id', inplace=True)
    
    return pred_df

In [ ]:
# Call function and check results
t = clv_cal_holdout.iloc[0,-1]  # set no of periods == duration_holdout
pred_purchases = predict_individual_purchases(bgf, clv, t)
display(pred_purchases.head())

In [ ]:
# Compare predictions to effective frequencies in holdout set
pred_evaluation = pd.concat([pred_purchases, 
                             clv_cal_holdout[['frequency_holdout']]], 
                            axis=1, 
                            sort=True
                            )

# For eval only look at customers with frequency value in holdout
pred_evaluation.dropna(how = 'any', inplace=True)
assert len(pred_evaluation) == len(clv_cal_holdout)

print("Pearson's R: ", round(pred_evaluation.corr().iloc[0,1],3))

In [ ]:
# Calculate difference of prediction to actual values
pred_evaluation['diff'] = np.abs(pred_evaluation['pred_purchases'] - pred_evaluation['frequency_holdout'])
print("Mean diff: ", pred_evaluation['diff'].sum() / len(pred_evaluation))

In [ ]:
# Display percentage of customers with respective difference ('error')
display(pred_evaluation['diff'].value_counts(dropna=False) / len(pred_evaluation))
sns.distplot(pred_evaluation['diff']);

In [ ]:
# Display the error (regression line should be 1 to 1)
sns.lmplot(x="pred_purchases", 
           y="frequency_holdout", 
           data=pred_evaluation, 
           x_estimator=np.mean,
           palette=['rebeccapurple', 'yellow']);

plt.xlim(0, 90)
plt.ylim(0, 90);

**Observation:** The correlation is very high, but the (linear) prediction constantly underestimates the effective purchases.

In [ ]:
# Trim outliers
pred_evaluation_trimmed = pred_evaluation.loc[
        pred_evaluation['frequency_holdout'] <= 30]

sns.lmplot(x="pred_purchases", 
           y="frequency_holdout", 
           data=pred_evaluation_trimmed, 
           x_estimator=np.mean,
           palette=['rebeccapurple', 'yellow']);

plt.xlim(0, 30)
plt.ylim(0, 30);

**Observation:** Without the frequency outliers the fit is a little better, but the prediction is still to low.

## 3) Estimate customer lifetime value using the Gamma-Gamma model
To estimate the CLV we use the [Gamma-Gamma submodel presented by Dr. Peter Fader and Dr. Bruce Hardie from Wharton](http://www.brucehardie.com/notes/025/gamma_gamma.pdf). It can be seen as an extension to the BG/NBD model, which focuses on modeling purchase count. Gamma-Gamma makes a few assumptions:
* At the customer level, the transaction/order value varies randomly around each customer’s average transaction value. (That, in itself, isn’t too controversial, but has to be checked, see below.)
* The observed mean value is an imperfect metric of the latent mean transaction value E(M), where M represents the monetary value.
* Average transaction value varies across customers, though these values are stationary. (This is a big assumption to make.)
* The distribution of average values across customers is independent of the transaction process. In other words, monetary value can be modeled separately from the purchase count and lifetime components of the model. This may or may not hold in typical business situations.

**IMPORTANT:** Fit only to returning customers (but you can predict for all)

In [ ]:
# Drop all one-time-only customers as clv can only be calculated for returning customers
returning_customers = clv.loc[clv['frequency'] > 0]
print(len(returning_customers))
display(returning_customers.head())

In [ ]:
# Check for independence between Frequency and Monetary Value
print("Pearson's r: ", round(np.corrcoef(returning_customers['monetary_value'],
                                         returning_customers['frequency'])[0][1], 2))

In [ ]:
# Fit Gamma-Gamma-Model on returning customers
ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returning_customers['frequency'],
        returning_customers['monetary_value'])

# Check results
print(ggf)

In [ ]:
# Predict CLV for a given period (in months) with bgf and ggf
clv_estimates = ggf.customer_lifetime_value(
    bgf,
    clv['frequency'],
    clv['recency'],
    clv['T'],
    clv['monetary_value'],
    time=4, # in months
    discount_rate=0.00)  # none for evaluation purposes

In [ ]:
# Check results
display(clv_estimates.head())

### Estimate average transaction values

This method computes the conditional expectation of the average profit per transaction for a group of one or more customers.

In [ ]:
avg_value = ggf.conditional_expected_average_profit(clv['frequency'],
                                                    clv['monetary_value'])

# check resutls
avg_value.head()

In [ ]:
sns.distplot(avg_value, bins=1500, color='rebeccapurple')
plt.xlim(0,1500);

**Observation:** The distribution differs quite a bit from the original values - see EDA notebook.

In [ ]:
# Compare mean of all avg_predictions vs effectively observed values
print("Expected conditional average value:", avg_value.mean()) # all customers
print("Observed average value:", returning_customers['monetary_value'].mean()) # returning only!

**Observation:** You cannot directly compare to monetary_value in clv_data, because the value of the first purchase is not calculated. From EDA I know that the average value per customer is 458 - so at least that is pretty close.

---

# APPENDIX - 2nd run: Calculations after removing Outliers

**Summary:"" I tested if outlier removal would improve the predictions, but it made the results worse.

In [ ]:
clean.count_outliers_IQR_method(clv, IQR_dist=2.5)

In [ ]:
clv_out = clean.remove_outliers_IQR_method(clv, IQR_dist=2.5)  # chose IQR-Dist of 2.5

In [ ]:
# Get indices / ids of outlier customers
ids_out = set(clv.index).difference(clv_out.index)
assert len(ids_out) == len(clv) - len(clv_out)
# Make a new transaction set without outliers
clv_data_out = clv_data[~clv_data['CustomerID'].isin(ids_out)]

In [ ]:
# Fit model with calibration and holdout sets on new data
summary_cal_holdout = calibration_and_holdout_data(clv_data_out, 'CustomerID', 'InvoiceDate', 
                                                   monetary_value_col='Sales',
                                                   calibration_period_end='2011-07-09',  # 7 months
                                                   observation_period_end='2011-12-09',  # 5 months
                                                   ) 

In [ ]:
# Check results
print(summary_cal_holdout.shape)
display(summary_cal_holdout.head())

In [ ]:
# 'Repair' duration_holdout: set to float - should not happen according to docs ...
import re
summary_cal_holdout['duration_holdout'] = \
    summary_cal_holdout['duration_holdout'].astype(str)    
summary_cal_holdout['duration_holdout'] = \
    summary_cal_holdout['duration_holdout'].apply(lambda x: int(re.findall('\d+', x)[0]))

In [ ]:
# Check results
display(summary_cal_holdout.head())

In [ ]:
# Fit on the _cal columns, and test on the _holdout columns"""
bgf_out = BetaGeoFitter(penalizer_coef=0.0)
bgf_out.fit(summary_cal_holdout['frequency_cal'], 
        summary_cal_holdout['recency_cal'], 
        summary_cal_holdout['T_cal'])

In [ ]:
# Plot results
plot_calibration_purchases_vs_holdout_purchases(bgf_out, 
                                                summary_cal_holdout, 
                                                figsize=(12,6),
                                                n=int(summary_cal_holdout['frequency_cal'].max() + 1),
                                                color=['rebeccapurple', 'yellow']
                                                );

In [ ]:
def predict_individual_purchases_float(model, df, t):
    """Predict number of purchases for period t for each customer
    and return them, rounded to int, in a DataFrame.
    """
    pred_list = []
    for customer in df.itertuples():
        pred_purchases = model.predict(t, 
                                     int(customer[1]), 
                                     int(customer[2]), 
                                     int(customer[3])
                                     )
        pred_list.append({'id': customer[0], 
                          'pred_purchases': pred_purchases})  # NOT round to int

    pred_df = pd.DataFrame(pred_list, columns=['id', 'pred_purchases'])
    pred_df.set_index('id', inplace=True)
    
    return pred_df

In [ ]:
# Call function and check results
t = summary_cal_holdout.iloc[0,-1]  # set no of periods == duration_holdout
pred_purchases = predict_individual_purchases(bgf_out, clv_out, t)
display(pred_purchases.head())

In [ ]:
# Compare predictions to effective frequencies in holdout set
pred_evaluation = pd.concat([pred_purchases, 
                            summary_cal_holdout[['frequency_holdout']]], 
                            axis=1, 
                            sort=True
                            )

# For eval only look at customers with frequency value in holdout
pred_evaluation.dropna(how = 'any', inplace=True)
assert len(pred_evaluation) == len(summary_cal_holdout)

print("Pearson's R: ", round(pred_evaluation.corr().iloc[0,1],3))

In [ ]:
pred_evaluation['diff'] = np.abs(pred_evaluation['pred_purchases'] - pred_evaluation['frequency_holdout'])
print("Mean diff: ", pred_evaluation['diff'].sum() / len(pred_evaluation))

In [ ]:
display(pred_evaluation['diff'].value_counts(dropna=False) / len(pred_evaluation))
sns.distplot(pred_evaluation['diff']);

### Estimate CLV

In [ ]:
returning_out = clv_out.loc[clv_out['frequency'] > 0]
print(len(returning_out))
display(returning_out.head())

In [ ]:
# Fit Gamma-Gamma-Model on returning customers
ggf_out = GammaGammaFitter(penalizer_coef = 0)
ggf_out.fit(returning_out['frequency'],
            returning_out['monetary_value'])

# Check results
print(ggf_out)

In [ ]:
# Predict CLV for a given period (in months) with bgf and ggf
clv_estimates_out = ggf_out.customer_lifetime_value(
    bgf_out,
    clv['frequency'],
    clv['recency'],
    clv['T'],
    clv['monetary_value'],
    time=4, # in months
    discount_rate=0.00)  # none for evaluation purposes

In [ ]:
avg_value_out = ggf_out.conditional_expected_average_profit(clv['frequency'],
                                                        clv['monetary_value'])

# check resutls
avg_value_out.head()

In [ ]:
sns.distplot(avg_value_out, bins=1500, color='rebeccapurple')
plt.xlim(0,1500);

In [ ]:
# Compare mean of all avg_predictions vs effectively observed values
print("Expected conditional average value:", avg_value_out.mean()) # all customers
print("Observed average value:", returning_out['monetary_value'].mean()) # returning only!

---